In [3]:
# Funciones básicas

def contar_valores(lista):
    conteo = []
    for valor in lista:
        existe = False
        for i in range(len(conteo)):
            if conteo[i][0] == valor:
                conteo[i][1] += 1
                existe = True
                break
        if not existe:
            conteo.append([valor, 1])
    return conteo

def maximo(mi_lista):
    conteo = contar_valores(mi_lista)
    max_valor = conteo[0][0]
    max_cuenta = conteo[0][1]
    for par in conteo:
        if par[1] > max_cuenta:
            max_valor = par[0]
            max_cuenta = par[1]
    return max_valor

def longitud(lista):
    contador = 0
    for _ in lista:
        contador += 1
    return contador

def raiz_cuadrada(n):
    x = n
    y = (x + 1) / 2
    e = 0.00001
    while (x - y) > e:
        x = y
        y = (x + n / x) / 2
    return y

def valor_absoluto(n):
    return -n if n < 0 else n

def pseudo_aleatorio(i, m):
    a = 17
    c = 43
    return (a * i + c) % m if m > 0 else 0


# Gini

def gini_impureza(labels):
    total = longitud(labels)
    if total == 0:
        return 0
    conteo = contar_valores(labels)
    impureza = 1
    for par in conteo:
        probabilidad = par[1] / total
        impureza -= probabilidad * probabilidad
    return impureza

# División del dataset

def dividir_dataset(data, labels, indice_feature, umbral):
    izquierda_datos = []
    izquierda_labels = []
    derecha_datos = []
    derecha_labels = []
    for i in range(longitud(data)):
        if data[i][indice_feature] <= umbral:
            izquierda_datos.append(data[i])
            izquierda_labels.append(labels[i])
        else:
            derecha_datos.append(data[i])
            derecha_labels.append(labels[i])
    return izquierda_datos, izquierda_labels, derecha_datos, derecha_labels

# Mejor división

def mejor_split(data, labels, features_usadas):
    mejor_ganancia = 0
    mejor_feature = None
    mejor_umbral = None
    gini_actual = gini_impureza(labels)

    for feat in features_usadas:
        valores = []
        for fila in data:
            v = fila[feat]
            if v not in valores:
                valores.append(v)

        for umbral in valores:
            izq_d, izq_l, der_d, der_l = dividir_dataset(data, labels, feat, umbral)
            p = longitud(izq_l) / longitud(labels)
            ganancia = gini_actual - p * gini_impureza(izq_l) - (1 - p) * gini_impureza(der_l)

            if ganancia > mejor_ganancia:
                mejor_ganancia = ganancia
                mejor_feature = feat
                mejor_umbral = umbral
    return mejor_feature, mejor_umbral

# Nodo del árbol

def crear_nodo(pred=None, feat=None, umbral=None, izq=None, der=None):
    return {
        'prediccion': pred,
        'feature': feat,
        'umbral': umbral,
        'izquierda': izq,
        'derecha': der
    }

# Construcción del árbol

def construir_arbol(data, labels, profundidad_max, profundidad=0):
    if longitud(contar_valores(labels)) == 1 or profundidad >= profundidad_max:
        pred = maximo(labels)
        return crear_nodo(pred=pred)

    num_features = longitud(data[0])
    k = 1
    raiz = raiz_cuadrada(num_features)
    while k * k < raiz * raiz:
        k += 1

    features = []
    for i in range(k):
        f = pseudo_aleatorio(i + profundidad, num_features)
        if f not in features:
            features.append(f)

    mejor_feat, mejor_umbral = mejor_split(data, labels, features)
    if mejor_feat is None:
        pred = maximo(labels)
        return crear_nodo(pred=pred)

    izq_d, izq_l, der_d, der_l = dividir_dataset(data, labels, mejor_feat, mejor_umbral)

    izq = construir_arbol(izq_d, izq_l, profundidad_max, profundidad + 1)
    der = construir_arbol(der_d, der_l, profundidad_max, profundidad + 1)

    return crear_nodo(feat=mejor_feat, umbral=mejor_umbral, izq=izq, der=der)

# Predicción con un árbol

def predecir_arbol(nodo, x):
    if nodo['prediccion'] is not None:
        return nodo['prediccion']
    if x[nodo['feature']] <= nodo['umbral']:
        return predecir_arbol(nodo['izquierda'], x)
    else:
        return predecir_arbol(nodo['derecha'], x)

# Clasificador Extra Trees

def extra_trees_fit(data, labels, num_arboles, profundidad_max):
    bosque = []
    n = longitud(data)
    for i in range(num_arboles):
        indices = [pseudo_aleatorio(i * 3 + j, n) for j in range(n)]
        datos_muestra = [data[k] for k in indices]
        etiquetas_muestra = [labels[k] for k in indices]
        arbol = construir_arbol(datos_muestra, etiquetas_muestra, profundidad_max)
        bosque.append(arbol)
    return bosque

def extra_trees_predict(bosque, x):
    votos = []
    for arbol in bosque:
        pred = predecir_arbol(arbol, x)
        votos.append(pred)
    return maximo(votos)

# -----------------------
# Ejemplo de uso
# -----------------------

mi_lista = [3, 1, 2, 3, 2, 1, 3, 10, 10, 10, 10, 10]
resultado = contar_valores(mi_lista)

for valor, frecuencia in resultado:
    print(f"Valor: {valor}, Frecuencia: {frecuencia}")

print(f"máximo valor", maximo(mi_lista))
print(f"longitud", longitud(mi_lista))

# Datos de entrenamiento
X = [
    [2, 3],
    [1, 5],
    [3, 6],
    [8, 1],
    [7, 2],
    [9, 0]
]
y = [0, 0, 0, 1, 1, 1]

# Entrenamiento y predicción
modelo = extra_trees_fit(X, y, 3, 3)
prediccion = extra_trees_predict(modelo, [2, 4])

print("Predicción:", prediccion)


Valor: 3, Frecuencia: 3
Valor: 1, Frecuencia: 2
Valor: 2, Frecuencia: 2
Valor: 10, Frecuencia: 5
máximo valor 10
longitud 12
Predicción: 0


In [4]:
# Árbol de decisión

# Función para dividir el conjunto de datos
def dividir_dataset(dataset, columna, valor):
    true_rows = []
    false_rows = []
    for fila in dataset:
        if fila[columna] >= valor:
            true_rows.append(fila)
        else:
            false_rows.append(fila)
    return true_rows, false_rows

# Conteo de clases en un conjunto
def contar_clases(dataset):
    resultados = {}
    for fila in dataset:
        etiqueta = fila[-1]
        if etiqueta not in resultados:
            resultados[etiqueta] = 0
        resultados[etiqueta] += 1
    return resultados

# Cálculo de entropía (medida de impureza)
def entropia(dataset):
    import math  # Puedes implementar log si no lo tienes disponible
    total = len(dataset)
    cuenta = contar_clases(dataset)
    impureza = 0
    for etiqueta in cuenta:
        p = cuenta[etiqueta] / total
        impureza -= p * math.log(p, 2)
    return impureza

# Nodo de decisión
class NodoDecision:
    def __init__(self, columna, valor, true_branch, false_branch):
        self.columna = columna
        self.valor = valor
        self.true_branch = true_branch
        self.false_branch = false_branch

# Nodo hoja
class Hoja:
    def __init__(self, dataset):
        self.predicciones = contar_clases(dataset)

# Función para encontrar la mejor división
def mejor_division(dataset):
    mejor_ganancia = 0
    mejor_columna = None
    mejor_valor = None
    mejor_division_resultado = None

    entropia_actual = entropia(dataset)
    n_columnas = len(dataset[0]) - 1

    for col in range(n_columnas):
        valores = set([fila[col] for fila in dataset])
        for val in valores:
            true_rows, false_rows = dividir_dataset(dataset, col, val)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            p = len(true_rows) / len(dataset)
            ganancia = entropia_actual - (p * entropia(true_rows) + (1 - p) * entropia(false_rows))
            if ganancia > mejor_ganancia:
                mejor_ganancia = ganancia
                mejor_columna = col
                mejor_valor = val
                mejor_division_resultado = (true_rows, false_rows)
    return mejor_ganancia, mejor_columna, mejor_valor, mejor_division_resultado

# Función recursiva para construir el árbol
def construir_arbol(dataset):
    ganancia, columna, valor, divisiones = mejor_division(dataset)
    if ganancia == 0:
        return Hoja(dataset)
    true_rows, false_rows = divisiones
    true_branch = construir_arbol(true_rows)
    false_branch = construir_arbol(false_rows)
    return NodoDecision(columna, valor, true_branch, false_branch)

# Clasificación de una nueva muestra
def clasificar(fila, nodo):
    if isinstance(nodo, Hoja):
        return nodo.predicciones
    if fila[nodo.columna] >= nodo.valor:
        return clasificar(fila, nodo.true_branch)
    else:
        return clasificar(fila, nodo.false_branch)

# Ejemplo de uso
datos = [
    [3, 'rojo', 'manzana'],
    [1, 'verde', 'manzana'],
    [2, 'rojo', 'uva'],
    [3, 'verde', 'manzana'],
    [2, 'verde', 'uva'],
    [1, 'rojo', 'uva']
]

# Para simplificar, transformamos valores de texto a números
# En MicroPython deberías hacer esto tú mismo o asumir datos ya numéricos

# Árbol
arbol = construir_arbol(datos)

# Clasificar una nueva fila
nueva_fila = [2, 'rojo']
print(clasificar(nueva_fila + ['?'], arbol))  # '?' es un marcador para la etiqueta


{'uva': 2}


In [5]:
# Aproximación del logaritmo base 2

def log2(x):
    # ln(x) ≈ (x - 1) - (x - 1)^2 / 2 + (x - 1)^3 / 3 - ...
    # ln(x)/ln(2) = log2(x)
    if x <= 0:
        return 0
    n = 10  # términos de la serie
    y = (x - 1) / (x + 1)
    y2 = y * y
    result = 0
    for i in range(n):
        term = (1 / (2 * i + 1)) * (y ** (2 * i + 1))
        result += term
    ln_x = 2 * result
    ln_2 = 0.69314718056  # constante aproximada
    return ln_x / ln_2

# Conteo de clases
def contar_clases(dataset):
    conteo = {}
    for fila in dataset:
        etiqueta = fila[-1]
        if etiqueta not in conteo:
            conteo[etiqueta] = 0
        conteo[etiqueta] += 1
    return conteo

# Entropía
def entropia(dataset):
    total = len(dataset)
    clases = contar_clases(dataset)
    impureza = 0
    for etiqueta in clases:
        p = clases[etiqueta] / total
        impureza -= p * log2(p)
    return impureza

# División de datos
def dividir_dataset(dataset, columna, valor):
    verdadero, falso = [], []
    for fila in dataset:
        if fila[columna] >= valor:
            verdadero.append(fila)
        else:
            falso.append(fila)
    return verdadero, falso

# Nodo hoja
class Hoja:
    def __init__(self, dataset):
        self.predicciones = contar_clases(dataset)

# Nodo de decisión
class NodoDecision:
    def __init__(self, columna, valor, rama_verdadera, rama_falsa):
        self.columna = columna
        self.valor = valor
        self.rama_verdadera = rama_verdadera
        self.rama_falsa = rama_falsa

# Mejor división
def mejor_division(dataset):
    mejor_ganancia = 0
    mejor_columna = 0
    mejor_valor = 0
    mejor_conjuntos = None
    incertidumbre = entropia(dataset)
    columnas = len(dataset[0]) - 1

    for col in range(columnas):
        valores = []
        for fila in dataset:
            if fila[col] not in valores:
                valores.append(fila[col])
        for val in valores:
            verdadero, falso = dividir_dataset(dataset, col, val)
            if len(verdadero) == 0 or len(falso) == 0:
                continue
            p = len(verdadero) / len(dataset)
            ganancia = incertidumbre - (p * entropia(verdadero) + (1 - p) * entropia(falso))
            if ganancia > mejor_ganancia:
                mejor_ganancia = ganancia
                mejor_columna = col
                mejor_valor = val
                mejor_conjuntos = (verdadero, falso)
    return mejor_ganancia, mejor_columna, mejor_valor, mejor_conjuntos

# Construcción del árbol
def construir_arbol(dataset):
    ganancia, columna, valor, conjuntos = mejor_division(dataset)
    if ganancia == 0:
        return Hoja(dataset)
    verdadero, falso = conjuntos
    rama_verdadera = construir_arbol(verdadero)
    rama_falsa = construir_arbol(falso)
    return NodoDecision(columna, valor, rama_verdadera, rama_falsa)

# Clasificación
def clasificar(fila, nodo):
    if isinstance(nodo, Hoja):
        return nodo.predicciones
    if fila[nodo.columna] >= nodo.valor:
        return clasificar(fila, nodo.rama_verdadera)
    else:
        return clasificar(fila, nodo.rama_falsa)

# DEMO DE PRUEBA

datos = [
    [2, 2, 1, 'E'],
    [1, 2, 0, 'M'],
    [0, 1, 2, 'A'],
    [2, 1, 2, 'E'],
    [1, 0, 1, 'A'],
    [2, 2, 2, 'E'],
    [0, 2, 1, 'M'],
    [1, 1, 1, 'M'],
    [2, 0, 2, 'A'],
    [0, 0, 0, 'A']
]


arbol = construir_arbol(datos)

# Clasificación de nuevo ejemplo
nueva = [2, 1]  # sin etiqueta
print(clasificar(nueva + ['?'], arbol))  # Agregamos '?' para simular etiqueta faltante


{'E': 3}


In [6]:
# Codificación manual one-hot
def codificar(v):
    if v == 'G': return [1, 0, 0]
    if v == 'R': return [0, 1, 0]
    if v == 'B': return [0, 0, 1]
    return [0, 0, 0]

# Datos
datos = [
    ["G", "G", "R", "E"],
    ["R", "G", "B", "M"],
    ["B", "R", "G", "A"],
    ["G", "R", "G", "E"],
    ["R", "B", "R", "A"],
    ["G", "G", "G", "E"],
    ["B", "G", "R", "M"],
    ["R", "R", "R", "M"],
    ["G", "B", "G", "A"],
    ["B", "B", "B", "A"]
]

# Separar X (características) y y (etiquetas)
X = []
y = []
for fila in datos:
    entrada = codificar(fila[0]) + codificar(fila[1]) + codificar(fila[2])
    X.append(entrada)
    y.append(fila[3])

# Aproximación de log base 2
def log2(x):
    if x <= 0: return 0
    y = (x - 1) / (x + 1)
    y2 = y * y
    s = 0
    i = 1
    while i < 20:
        s += (y ** (2 * i - 1)) / (2 * i - 1)
        i += 1
    ln_x = 2 * s
    ln_2 = 0.693147
    return ln_x / ln_2

# Entropía de un conjunto de etiquetas
def entropia(clases):
    total = len(clases)
    if total == 0: return 0
    frec = {}
    for c in clases:
        if c not in frec: frec[c] = 0
        frec[c] += 1
    e = 0
    for v in frec.values():
        p = v / total
        e -= p * log2(p)
    return e

# Dividir por columna y valor
def dividir(X, y, col, val):
    X_v, y_v, X_f, y_f = [], [], [], []
    for i in range(len(X)):
        if X[i][col] == val:
            X_v.append(X[i])
            y_v.append(y[i])
        else:
            X_f.append(X[i])
            y_f.append(y[i])
    return X_v, y_v, X_f, y_f

# Buscar la mejor división
def mejor_division(X, y):
    base = entropia(y)
    mejor_gan = 0
    mejor_col = -1
    mejor_val = None
    mejor_sets = None
    n_cols = len(X[0])
    for col in range(n_cols):
        valores = []
        for fila in X:
            if fila[col] not in valores:
                valores.append(fila[col])
        for val in valores:
            X_v, y_v, X_f, y_f = dividir(X, y, col, val)
            if len(y_v) == 0 or len(y_f) == 0:
                continue
            p = len(y_v) / len(y)
            gan = base - p * entropia(y_v) - (1 - p) * entropia(y_f)
            if gan > mejor_gan:
                mejor_gan = gan
                mejor_col = col
                mejor_val = val
                mejor_sets = (X_v, y_v, X_f, y_f)
    return mejor_gan, mejor_col, mejor_val, mejor_sets

# Definir nodo hoja
class Hoja:
    def __init__(self, y):
        self.pred = {}
        for c in y:
            if c not in self.pred:
                self.pred[c] = 0
            self.pred[c] += 1

# Definir nodo de decisión
class Nodo:
    def __init__(self, col, val, si, no):
        self.col = col
        self.val = val
        self.si = si
        self.no = no

# Construir árbol recursivamente
def construir(X, y):
    gan, col, val, sets = mejor_division(X, y)
    if gan == 0:
        return Hoja(y)
    X_si, y_si, X_no, y_no = sets
    rama_si = construir(X_si, y_si)
    rama_no = construir(X_no, y_no)
    return Nodo(col, val, rama_si, rama_no)

# Clasificar nueva muestra
def clasificar(fila, nodo):
    if isinstance(nodo, Hoja):
        return nodo.pred
    if fila[nodo.col] == nodo.val:
        return clasificar(fila, nodo.si)
    else:
        return clasificar(fila, nodo.no)

# Entrenar árbol
arbol = construir(X, y)

# Clasificar nueva entrada: ["R", "G", "G"]
nueva = codificar("R") + codificar("G") + codificar("G")
prediccion = clasificar(nueva, arbol)

# Mostrar predicción
print("Resultado:", prediccion)


Resultado: {'M': 2}
